In [ ]:
# import necassary libriries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# load dataset
df_statistics = pd.read_csv('../data/statistics_20210707.csv')

In [ ]:
# Convert column `yearmonth` to datetime
df_statistics['yearmonth'] = pd.to_datetime(df_statistics['yearmonth'])

In [ ]:
# Heatmap
corr = df_statistics.corr()
ax = sns.heatmap(
    corr,
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [ ]:
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
# Plot
plt.figsize
sns.displot(data=df_statistics, x='yearmonth', y='inquiry_count')